In [ ]:
!pip install pyspark findspark

     |████████████████████████████████| 281.4 MB 30 kB/s 
     |████████████████████████████████| 198 kB 11.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=aee9d8add59e54b61a86b90c6c3dc92f989107bc244b341ad8b1ea82bb06f624
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
import findspark
findspark.init()
import pyspark

from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession


sp = SparkContext()
ss = SparkSession(sp).builder.config("spark.driver.memory", "6g").getOrCreate()

import pyspark.sql.functions as f

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/BigDataManagement/')

In [ ]:
files_list = ['drive/MyDrive/BigDataManagement/weather-data/out_2016.txt', 
              'drive/MyDrive/BigDataManagement/weather-data/out_2017.txt',
              'drive/MyDrive/BigDataManagement/weather-data/out_2018.txt',
              'drive/MyDrive/BigDataManagement/weather-data/out_2019.txt',
              'drive/MyDrive/BigDataManagement/weather-data/out_2020.txt']
df2 = ss.read.csv(files_list, header=False)
df2 = df2.filter(df2._c0 != 'STATION_ID')

In [ ]:
split_col = f.split(df2['_c3'], ' ')

df2 = df2.withColumn('wind_angle', split_col.getItem(0))
df2 = df2.withColumn('wind_qa', split_col.getItem(1))
df2 = df2.withColumn('wind_type', split_col.getItem(2))
df2 = df2.withColumn('wind_speed', split_col.getItem(3))
df2 = df2.withColumn('wind_qs', split_col.getItem(4))

In [ ]:
split_col = f.split(df2['_c4'], ' ')

df2 = df2.withColumn('tmp_val', split_col.getItem(0))
df2 = df2.withColumn('tmp_q', split_col.getItem(1))

In [ ]:
split_col = f.split(df2['_c5'], ' ')

df2 = df2.withColumn('dew_val', split_col.getItem(0))
df2 = df2.withColumn('dew_q', split_col.getItem(1))

In [ ]:
split_col = f.split(df2['_c6'], ' ')

df2 = df2.withColumn('slp_val', split_col.getItem(0))
df2 = df2.withColumn('slp_q', split_col.getItem(1))

In [ ]:
df2 = df2.drop('_c2', '_c3', '_c4', '_c5', '_c6', '_c7', '_c8', '_c9', '_c10', '_c11', '_c12', '_c13')

In [ ]:
df2.show()

+-----------+----------+----------+-------+---------+----------+-------+-------+-----+-------+-----+-------+-----+
|        _c0|       _c1|wind_angle|wind_qa|wind_type|wind_speed|wind_qs|tmp_val|tmp_q|dew_val|dew_q|slp_val|slp_q|
+-----------+----------+----------+-------+---------+----------+-------+-------+-----+-------+-----+-------+-----+
|06022499999|2016-01-01|       210|      1|        N|      0113|      1|  +0070|    1|  +0060|    1|  99999|    9|
|06022499999|2016-01-02|       130|      1|        N|      0118|      1|  +0070|    1|  +0030|    1|  99999|    9|
|06022499999|2016-01-03|       110|      1|        N|      0221|      1|  +0020|    1|  -0010|    1|  99999|    9|
|06022499999|2016-01-04|       110|      1|        N|      0211|      1|  +0020|    1|  +0010|    1|  99999|    9|
|06022499999|2016-01-05|       110|      1|        N|      0190|      1|  +0020|    1|  -0020|    1|  99999|    9|
|06022499999|2016-01-06|       110|      1|        N|      0170|      1|  +0010|

In [ ]:
df2 = df2.withColumnRenamed('_c0', 'station_id').withColumnRenamed('_c1', 'date')

In [ ]:
df2 = df2.dropDuplicates(['station_id', 'date'])

----

Formatting the columns and dropping the extensive columns

In [ ]:
df2 = df2.withColumn("dew_val", df2.dew_val.cast('int'))
df2 = df2.withColumn("tmp_val", df2.tmp_val.cast('int'))

----

Sort the values (not nulls)

In [ ]:
dew_vals = [row.dew_val for row in df2.select('dew_val').distinct().collect()]
tmp_vals = [row.tmp_val for row in df2.select('tmp_val').distinct().collect()]

In [ ]:
dew_vals.sort()

In [ ]:
dew_vals[-5:]

[10294, 10302, 10318, 10333, 99999]

In [ ]:
tmp_vals.sort()

In [ ]:
tmp_vals[-10:]

[587, 590, 591, 597, 599, 600, 602, 603, 615, 9999]

In [ ]:
len(tmp_vals)

1356

In [ ]:
df3 = df2.select(['station_id', 'date', 'tmp_val', 'dew_val']).filter((df2.tmp_val != 9999) & (df2.dew_val != 99999) & (df2.dew_val != 9999))
df3.cache()
df2.unpersist()

DataFrame[station_id: string, date: string, wind_angle: string, wind_qa: string, wind_type: string, wind_speed: string, wind_qs: string, tmp_val: int, tmp_q: string, dew_val: int, dew_q: string, slp_val: string, slp_q: string]

----

In [ ]:
import pandas as pd

x = pd.read_csv('/content/drive/MyDrive/BigDataManagement/stocks-ta.csv', header = 0, index_col = 0)
x.head()

,stock_name,date,volume,price_ff,price_bf,price
0,1004.Asien--Australien--Südamerika--Afrika_Aus...,2016-01-01,185675.0,57.50,57.50,57.50
1,1004.Asien--Australien--Südamerika--Afrika_Aus...,2016-01-02,NaN,57.50,57.80,57.50
2,1004.Asien--Australien--Südamerika--Afrika_Aus...,2016-01-03,NaN,57.50,57.80,57.50
3,1004.Asien--Australien--Südamerika--Afrika_Aus...,2016-01-04,305400.0,57.80,57.80,57.80
4,1004.Asien--Australien--Südamerika--Afrika_Aus...,2016-01-05,270032.0,57.17,57.17,57.17


In [ ]:
dates = x.date.unique()
dates.sort()
len(dates)

1000

----

In [ ]:
stations = [row.station_id for row in df3.filter(df3.date.isin(list(dates))).groupBy('station_id')\
                                                                      .count().orderBy('count', ascending=False)\
                                                                      .take(1000)]

In [ ]:
dates = list(dates)
dates.sort()

In [ ]:
dates[-1]

'2019-02-21'

In [ ]:
df_small = df3.filter(df3.station_id.isin(stations))\
             .filter((dates[0] <= df3.date) & (df3.date <= dates[-1]))

In [ ]:
df_small = df_small.cache()
df3.unpersist()

DataFrame[station_id: string, date: string, tmp_val: int, dew_val: int]

In [ ]:
df_small= df_small.withColumn('date', f.col('date'))\
                  .withColumn("date_existent", f.col("date"))

----

In [ ]:
from pyspark.sql.types import ArrayType, StringType

In [ ]:
def func():
    return dates

In [ ]:
func_udf = f.udf(func, ArrayType(StringType()))

In [ ]:
df_base = df_small.select('station_id').distinct().withColumn('date', f.explode(func_udf()))\
                                               .withColumn('date', f.col('date'))

In [ ]:
df_full = df_base.join(df_small, ['station_id', 'date'], "leftouter")

In [ ]:
df_small.unpersist()
df_full = df_full.cache()

In [ ]:
df_full.count()

1000000

In [ ]:
df_full.where(df_full.dew_val.isNull()).count()

1202

In [ ]:
df_full.where(df_full.tmp_val.isNull()).count()

1202

----

In [ ]:
from pyspark.sql import Window
import sys

In [ ]:
window_ff = Window.partitionBy('station_id')\
               .orderBy('date')\
               .rowsBetween(-sys.maxsize, 0)
               
window_bf = Window.partitionBy('station_id')\
               .orderBy('date')\
               .rowsBetween(0, sys.maxsize)

In [ ]:
date_last = f.last(df_full['date_existent'], ignorenulls=True).over(window_ff)
date_next = f.first(df_full['date_existent'], ignorenulls=True).over(window_bf)

tmp_last = f.last(df_full['tmp_val'], ignorenulls=True).over(window_ff)
tmp_next = f.first(df_full['tmp_val'], ignorenulls=True).over(window_bf)

dew_last = f.last(df_full['dew_val'], ignorenulls=True).over(window_ff)
dew_next = f.first(df_full['dew_val'], ignorenulls=True).over(window_bf)

df_filled = df_full.withColumn('dew_ff', dew_last)\
                   .withColumn('dew_bf', dew_next)\
                   .withColumn('tmp_ff', tmp_last)\
                   .withColumn('tmp_bf', tmp_next)\
                   .withColumn('date_ff', date_last)\
                   .withColumn('date_bf', date_next)

In [ ]:
df_filled.where(df_filled.date_existent.isNull()).show()

+-----------+----------+-------+-------+-------------+------+------+------+------+----------+----------+
| station_id|      date|tmp_val|dew_val|date_existent|dew_ff|dew_bf|tmp_ff|tmp_bf|   date_ff|   date_bf|
+-----------+----------+-------+-------+-------------+------+------+------+------+----------+----------+
|01016099999|2018-11-19|   null|   null|         null|  -143|   -90|   -97|   -64|2018-11-16|2018-11-22|
|01016099999|2018-11-20|   null|   null|         null|  -143|   -90|   -97|   -64|2018-11-16|2018-11-22|
|01016099999|2018-11-21|   null|   null|         null|  -143|   -90|   -97|   -64|2018-11-16|2018-11-22|
|02520099999|2018-11-19|   null|   null|         null|    65|   -30|    65|   -10|2018-11-16|2018-11-22|
|02520099999|2018-11-20|   null|   null|         null|    65|   -30|    65|   -10|2018-11-16|2018-11-22|
|02520099999|2018-11-21|   null|   null|         null|    65|   -30|    65|   -10|2018-11-16|2018-11-22|
|02741099999|2018-11-19|   null|   null|         null| 

In [ ]:
from pyspark.sql.types import IntegerType
import datetime

def interpol(x, x_prev, x_next, y_prev, y_next, y):
    if x_prev == x_next:
        return int(y)
    else:
        #if (y_prev is not None) & (y_next is not None): 
        #    m = (float(y_next)-float(y_prev))/(datetime.date(int(x_next[:3]), int(x_next[5:7]), int(x_next[-2:])) - datetime.date(int(x_prev[:3]), int(x_prev[5:7]), int(x_prev[-2:]))).days
        #    y_interpol = float(y_prev) + m * (datetime.date(int(x[:3]), int(x[5:7]), int(x[-2:])) - datetime.date(int(x_prev[:3]), int(x_prev[5:7]), int(x_prev[-2:]))).days
        if (y_prev is not None) & (y_next is not None): 
            m = (int(y_next)-int(y_prev))/(datetime.datetime.strptime(x_next, '%Y-%m-%d') - datetime.datetime.strptime(x_prev, '%Y-%m-%d')).days
            y_interpol = int(y_prev) + m * (datetime.datetime.strptime(x, '%Y-%m-%d') - datetime.datetime.strptime(x_prev, '%Y-%m-%d')).days
        if x_prev is not None:
            y_interpol = y_prev
        else:
            y_interpol = y_next
        y_interpol
        return int(round(y_interpol))

interpol_udf = f.udf(interpol, IntegerType())

In [ ]:
df_filled = df_filled.withColumn('tmp_interpol',
                                 interpol_udf('date', 'date_ff', 'date_bf', 'tmp_ff', 'tmp_bf', 'tmp_val'))\
                     .withColumn('dew_interpol',
                                 interpol_udf('date', 'date_ff', 'date_bf', 'dew_ff', 'dew_bf', 'dew_val'))\
                     .drop('date_existent', 'date_ff', 'date_bf',
                           'dew_val', 'dew_bf', 'dew_ff',
                           'tmp_val', 'tmp_bf', 'tmp_ff')\
                     .withColumnRenamed('dew_interpol', 'dew_val')\
                     .withColumnRenamed('tmp_interpol', 'tmp_val')
df_filled.repartition(1000)

DataFrame[station_id: string, date: string, tmp_val: int, dew_val: int]

In [ ]:
df_filled.where(df_filled.tmp_val.isNull()).count()

0

In [ ]:
df_filled.where(df_filled.dew_val.isNull()).count()

0

In [ ]:
df_filled.count()

1000000

In [ ]:
df_filled.repartition(1).toPandas().to_csv('/content/drive/MyDrive/BigDataManagement/weather-ta2.csv')

In [ ]:
df_filled.show()

+-----------+----------+-------+-------+
| station_id|      date|tmp_val|dew_val|
+-----------+----------+-------+-------+
|01408099999|2016-01-01|     73|     41|
|01408099999|2016-01-02|     24|     -3|
|01408099999|2016-01-03|     -9|    -90|
|01408099999|2016-01-04|    -18|    -99|
|01408099999|2016-01-05|    -41|   -106|
|01408099999|2016-01-06|    -29|    -92|
|01408099999|2016-01-07|    -38|   -105|
|01408099999|2016-01-08|    -31|    -96|
|01408099999|2016-01-09|    -36|    -91|
|01408099999|2016-01-10|    -14|    -21|
|01408099999|2016-01-11|     44|    -18|
|01408099999|2016-01-12|      8|    -48|
|01408099999|2016-01-13|      1|    -15|
|01408099999|2016-01-14|      8|     -6|
|01408099999|2016-01-15|     -3|    -39|
|01408099999|2016-01-16|     15|    -39|
|01408099999|2016-01-17|    -69|    -85|
|01408099999|2016-01-18|    -23|    -34|
|01408099999|2016-01-19|     -8|    -18|
|01408099999|2016-01-20|    -96|   -105|
+-----------+----------+-------+-------+
only showing top